In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches
import scipy.spatial as spatial
from random import randint
import random

In [ ]:
data_set = 'yolo'

if data_set == 'iris':
    from sklearn import datasets
    iris = datasets.load_iris()
    x = iris.data[:, :2]
    df = pd.DataFrame().from_dict({'x': x[:,0], 'y': x[:,1]})
    # DBSCAN parameter
    eps = 0.2
    min_pts = 8
else: 
    df = pd.read_csv('https://raw.githubusercontent.com/lnxdxC/DSAI/main/L03_Clustering/DBSCAN_data.csv')
    # DBSCAN parameter
    eps = 2
    min_pts = 3
    
# Search tree
search_tree = spatial.cKDTree(np.c_[df.x, df.y])

In [ ]:
# Allocate new channels to store cluster information
df['id'] = np.nan
df['color'] = '#000000'

In [ ]:
# Show the data set
df.plot.scatter(x='x', y='y', s=50, alpha=0.5);

# DBSCAN
Density-Based Spatial Clustering of Applications with Noise (DBSCAN) has 2 core parameters which have to be assigned before we can start: 
1. $\epsilon$
2. ```min_pts```

Here $\epsilon$ represents the radius of a circle around a particular point within the dataset and ```min_pts``` denotes a threshold used in the clustering process to distinguish between a cluster, a border, and a noise point.

In [ ]:
def get_points_within_circle(df, idx, eps, search_tree=search_tree):
    """
    We have to find all elements around a given centroid, within given radius epsilon. \n
    For efficient code, we use kd-tree search.
    
    :param df: data frame
    :param idx: actual index of the data set
    :param eps: radius of the circle
    :param search_tree: kd-tree 
    """
    
    return search_tree.query_ball_point([df.x[idx],df.y[idx]], eps)

In [ ]:
def get_point_type(df, point_types, idx, circle_points, min_pts=3):
    """
    Based on the amount of points within the circle, we assign either border, core, or noise point at a given position
    
    :param df: data frame
    :param point_types: variable to store the information of the point types
    :param idx: actual index of the data set
    :param circle_points: points within the actual circle
    :param min_pts: amount of points to be within the circle to assign core point to pt[idx]
    """    
    if len(circle_points) - 1 == 0:  # We have to subtract one, since our centroid is also included in circle_points
        point_types['noise'].append(idx)
    else:
        n_pts = len(circle_points) - 1

        if n_pts >= min_pts:
            point_types['core'].extend(circle_points)

        elif (n_pts < min_pts) and (n_pts != 0):
            point_types['border'].append(idx)

    return point_types, df

In [ ]:
def update_class(df, circle_points, key='id', range=75):
    """
    Update the class affiliation. If at least one element within the class has already a cluster ID, we assign all members of the circle to that cluster.  \n
    Additionally, we have to look up in the data frame to all
    """
    # If any value is nan, we assign a random number to it
    if len(circle_points) == 1:
        return df

    if all(df[key][circle_points].isna()):
        df['id'][circle_points] = np.random.randint(1, range)
    else:
        # Set all values in the df to the max value if matching within circle
        for item in df[key][circle_points].unique():
            if np.isnan(item):
                continue
            df['id'][circle_points] = df[key][circle_points].max()
            
            # Seed a random value to get new colors
            random.seed()
            df['color'][df['id'] == item] = '#%06X' % randint(0, 0xFFFFFF)

    return df

In [ ]:
point_types = {'core':   [],
               'border': [],
               'noise':  []}

# Create figure
fig, ax = plt.subplots(figsize=(10, 10))

draw_result = False
break_at = 2

for idx, _ in enumerate(df.x):
    
    circle_points = get_points_within_circle(df, idx, eps, search_tree)

    point_types, df = get_point_type(df, point_types, idx, circle_points, min_pts)

    df = update_class(df, circle_points)
    
    if draw_result:
        ax.scatter(df.x, df.y, color=df.color, s=50)

        # Draw current point
        ax.scatter(df.x[idx], df.y[idx], marker='h', label='current point', s=50)
        # Draw circle around current point
        circle = plt.Circle((df.x[idx], df.y[idx]), eps, alpha=0.3, edgecolor='k')
        ax.add_patch(circle)

        # Draw points within the circle
        ax.scatter(df.x[circle_points], df.y[circle_points], color='b', s=50)
        ax.scatter(df.x[idx], df.y[idx], color='m', marker='x', s=50)

        # Draw noise points
        ax.scatter(df.x[point_types['noise']], df.y[point_types['noise']], marker='*', color='r', s=50)

        # Draw border points
        ax.scatter(df.x[point_types['border']], df.y[point_types['border']], color='m', s=50)
        
        if idx == break_at:
            break
            
ax.scatter(df.x, df.y, color=df.color, s=50)

# sklearn implementation

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
# Since sklearn requires data in (n,2)-shape, we have to prepare them
X = np.array([df.x, df.y]).T

In [ ]:
db = DBSCAN(eps=eps, min_samples=min_pts).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

fig, ax = plt.subplots(2, figsize=(10, 10))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    ax[0].plot( xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    ax[0].plot(xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=6)
ax[1].scatter(df.x, df.y, color=df.color, s=50)  
plt.show()

## Large scale DBSCAN

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lnxdxC/DSAI/main/L03_Clustering/t48k.csv')
X = np.array([df.x, df.y]).T

In [ ]:
X = np.array([df.x, df.y]).T

In [ ]:
db = DBSCAN(eps=8, min_samples=15).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

fig, ax = plt.subplots(figsize=(10, 5))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    ax.plot( xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    ax.plot(xy[:, 0], xy[:, 1], "o", markerfacecolor=tuple(col), markeredgecolor="k", markersize=6)
plt.show()